# Health Analysis

In [ ]:
%run analysis_functions.ipynb

In [ ]:
df, arguments = read_clean_data()
sheet_id = arguments["sheet_id"]
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}"

<hr style="height:2px;border-width:0;color:black;background-color:black">

## Cervical mucus charting

In [ ]:
# Generate to save, but don't display
chart = chart_cervical_mucus(
    df,
    with_text=False,
    outfile="charting",
)

### ![](charting.png)

In [ ]:
chart = chart_cervical_mucus(
    df,
    with_text=True,
    outfile="charting_with_text",
)

<hr style="height:2px;border-width:0;color:black;background-color:black">

## Current symptoms
- Severe fatigue
- Daily headaches
- Pelvic pain
- Pain with sex
- Abdominal pain and bloating
- Nausea, frequent and loose BMs
- Stomach pain when empty, acid reflux
- Painful and swollen finger joints
- Frequent urination, painful bladder
- Occasional hypoglycemia: shaky and weak if I miss a meal
- Tinnitus

## Symptoms, quantified

In [ ]:
numeric_metrics = [
    "BMs",
    "previous night Oura score",
    "steps",
]

categorical_metrics = [
    "pelvic pain",
    "stomach pain",
    "fatigue",
    "headache",
    "uncomfortable stomach",
    "bloated stomach_bool",
    "diarrhea_bool",
    "nausea",
    "white clouds in urine",
]

In [ ]:
# last N days, excluding most recent, incomplete day
last_n_days = 30
start_dt = df[- last_n_days - 1: -1].index.min()
end_dt = df[- last_n_days - 1: -1].index.max()
min_pain = 4

summary = generate_summary(
    df,
    start_dt,
    end_dt,
    categorical_metrics,
    numeric_metrics,
    min_pain=min_pain,
)
print(summary)

In [ ]:
# Right before second surgery
start_dt = datetime.datetime(2023, 9, 1)
end_dt = datetime.datetime(2023, 9, 26)

<hr style="height:2px;border-width:0;color:black;background-color:black">

## Medications

In [ ]:
# second to last day, since today likely isn't over yet
end_dt = df.index[-2]
n_days = 1

summary = generate_med_summary(
    df,
    end_dt,
    n_days,
)
print(summary)

In [ ]:
plot_meds_over_time(df)

In [ ]:
plot_diet_metric_by_color(
    df=df,
    y_metric="BMs",
    color_metric="diarrhea_bool",
)

In [ ]:
diets = ["low inflammation", "elimination", "adding back", "everything"]
df, metric_data, diet_data, metrics_dict = get_diet_data(df, diets)
start_dt = pd.datetime(2024, 1, 1)
end_dt = df.index[-2]
rolling_days = 7

metrics = [
    "BMs",
]
for metric in metrics:
    plot_diet_metric(
        metric,
        df,
        diet_data,
        start_dt,
        end_dt,
        rolling_days=rolling_days,
        include_scatter=True,
    )

In [ ]:
## T-tests comparing diets

# control_diet = "elimination"
# other_diets = ["low inflammation", "adding back"]

# # remove transition period, before diet can have an effect
# transition_days = 14

# for other_diet in other_diets:
#     print(f"{other_diet} vs. {control_diet}")
    
#     control = df[df["diet name"] == control_diet][transition_days:]
#     other = df[df["diet name"] == other_diet][transition_days:]

#     for metric in metrics:
#         stat, p = ttest_ind(
#             other[metric].dropna(),
#             control[metric].dropna(),
#         )
#         ratio = other[metric].mean() / control[metric].mean()
        
#         metric_str = metric.replace("_bool", "")
#         print(f"{metric_str}: {round(stat, 2)}, p={round(p, 3)}")

#     print()